<a href="https://colab.research.google.com/github/devloic/piper-colab-train-notebook/blob/main/piper_colab_train_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
def mount_with_drive_mount():
  !mkdir -p /content/drive/drive.mount/
  from google.colab import drive
  drive.mount('/content/drive/drive.mount/', force_remount=True)

mount_with_drive_mount()

Mounted at /content/drive/drive.mount/


In [ ]:
!pip install fastapi uvicorn nest-asyncio -q
!npm install -g localtunnel

In [ ]:
  #pure python
  import subprocess
  import os
  import time
  import requests

  # Change to app directory
  os.chdir("/content/drive/drive.mount/MyDrive/piper-colab-train-fastapi")

  # Clean up
  subprocess.run(['pkill', '-f', 'uvicorn app:app'], capture_output=True)
  subprocess.run(['pkill', '-f', 'localtunnel'], capture_output=True)
  for f in ['/content/logs.txt', '/content/lt.txt']:
      if os.path.exists(f): os.remove(f)

  # Start FastAPI
  print("Starting FastAPI...")
  with open('/content/logs.txt', 'w') as log:
      fastapi = subprocess.Popen(
          ['uvicorn', 'app:app', '--host', '0.0.0.0', '--port', '8501'],
          stdout=log, stderr=subprocess.STDOUT
      )

  # Wait for ready
  time.sleep(3)

  # Start localtunnel
  print("Starting localtunnel...")
  lt = subprocess.Popen('npx localtunnel --port 8501 > /content/lt.txt 2>&1',
  shell=True)

  # Wait for URL
  print("Waiting for tunnel URL...")
  for i in range(30):
      if os.path.exists('/content/lt.txt'):
          with open('/content/lt.txt') as f:
              content = f.read()
              if 'your url is:' in content.lower():
                  url = content.split('your url is:')[1].split('\n')[0].strip()
                  print(f"\n🚀  App: {url}")
                  print(f"📚  Docs: {url}/docs")
                  try:
                      pwd = requests.get('https://loca.lt/mytunnelpassword').text.strip()
                      print(f"🔑  Password: {pwd}")
                  except: pass
                  break
      time.sleep(1)